In [3]:
api_key = '11ad52b274313ff63fe850abfd31dc09ee8b0575'

In [4]:
import requests
from io import BytesIO
import zipfile
import xmltodict
import json


In [5]:
BASE_PARAM={'crtfc_key': api_key}

In [6]:
company_info = 'https://opendart.fss.or.kr/api/company.json'

In [7]:
param = dict(BASE_PARAM)
param['corp_code'] = '00126186'

In [8]:
res = requests.get(company_info, params = param).json()

In [9]:
overview =res

In [15]:
from yahoofinancials import YahooFinancials
from datetime import datetime, timedelta
import pandas as pd

In [31]:
INTERVAL = 100

now = datetime.now()
end = now.strftime('%Y-%m-%d')
start = (now - timedelta(days = INTERVAL *7)).strftime('%Y-%m-%d')

In [32]:
stock_code = overview['stock_code'] + '.KS'
yahoo_financials = YahooFinancials(stock_code)

In [33]:
data =yahoo_financials.get_historical_price_data(start_date = start,
    end_date = end, time_interval = 'daily')
data

{'018260.KS': {'eventsData': {'dividends': {'2020-12-29': {'amount': 2400.0,
     'date': 1609200000,
     'formatted_date': '2020-12-29'}}},
  'firstTradeDate': {'formatted_date': '2014-08-25', 'date': 1408924800},
  'currency': 'KRW',
  'instrumentType': 'EQUITY',
  'timeZone': {'gmtOffset': 32400},
  'prices': [{'date': 1577923200,
    'high': 195000.0,
    'low': 192000.0,
    'open': 194500.0,
    'close': 192000.0,
    'volume': 59743,
    'adjclose': 189411.234375,
    'formatted_date': '2020-01-02'},
   {'date': 1578009600,
    'high': 195000.0,
    'low': 191500.0,
    'open': 193500.0,
    'close': 192500.0,
    'volume': 66452,
    'adjclose': 189904.484375,
    'formatted_date': '2020-01-03'},
   {'date': 1578268800,
    'high': 192500.0,
    'low': 190000.0,
    'open': 191000.0,
    'close': 190500.0,
    'volume': 64597,
    'adjclose': 187931.453125,
    'formatted_date': '2020-01-06'},
   {'date': 1578355200,
    'high': 193000.0,
    'low': 191000.0,
    'open': 19200

In [34]:
stock_df = pd.DataFrame( data[stock_code]['prices'])
stock_df

,date,high,low,open,close,volume,adjclose,formatted_date
0,1577923200,195000.0,192000.0,194500.0,192000.0,59743,189411.234375,2020-01-02
1,1578009600,195000.0,191500.0,193500.0,192500.0,66452,189904.484375,2020-01-03
2,1578268800,192500.0,190000.0,191000.0,190500.0,64597,187931.453125,2020-01-06
3,1578355200,193000.0,191000.0,192000.0,192500.0,51281,189904.484375,2020-01-07
4,1578441600,190500.0,188000.0,190500.0,188500.0,91742,185958.421875,2020-01-08
...,...,...,...,...,...,...,...,...
471,1637884800,154000.0,149000.0,153000.0,150000.0,123815,150000.000000,2021-11-26
472,1638144000,150500.0,146500.0,147000.0,147500.0,94722,147500.000000,2021-11-29
473,1638230400,150500.0,142500.0,148000.0,143500.0,249077,143500.000000,2021-11-30
474,1638316800,149500.0,143500.0,144000.0,147500.0,109850,147500.000000,2021-12-01


In [35]:
stock_df['formatted_date'] = stock_df['formatted_date'].apply(lambda x: x.replace('-', '/'))

In [36]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [48]:
stock_fig = make_subplots(specs = [[{'secondary_y':True}]])

In [49]:
stock_fig.add_trace(go.Bar(x = stock_df['formatted_date'],
                          y = stock_df['volume'],
                          name = '거래량',
                          opacity = 0.3), secondary_y = True),

stock_fig.add_trace(go.Scatter(x = stock_df['formatted_date'],
                          y = stock_df['close'],
                          name = '주가'), secondary_y = False)

stock_fig.update_layout(title_text='<b>삼성SDS 주가 추이</b><br><sup>(단위: 원, 주)</sup>')